# MEF en 1D con condiciones de Robin

Nota: a este documento no lo podremos ejecutar de manera dinámica con Binder, sólo lo podrán correr si tienen instalado:

- Python 3.5 o más nuevo instalado.
- [Jupyter Notebook](https://jupyter.readthedocs.io/en/latest/install.html).
- [FEniCS](https://fenicsproject.org/).


## Resolvemos la ecuación de Poisson

En este tutorial desarrollaremos el método en su versión unidimensional para la siguiente ecuación:

$$ \left \{ \begin{array}{l} -\left(au^{'}\right)^{'}=f \ \ \text{ para } \ \ x\in I=\left(0,1 \right) \\   au^{'}(0)=k_{0}\left(u(0)-g_{0}\right)  \ \ \text{condición de borde Robin} \\  -au^{'}(1)=k_{1}\left(u(1)-g_{1}\right)  \ \ \text{condición de borde Robin} \end{array} \right . \tag{1}$$

Si observamos las diferencias con el tutorial del [problema de Dirichlet](https://nbviewer.jupyter.org/github/rirastorza/Intro2FEM/blob/master/Elementos_finitos_en_1D/fem1D.ipynb), en este caso aparecen nuevas constantes: $a$, $k_{i}$, y $g_{i}$ (con $i=$ 0, 1). Estas constantes pueden significar varias cosas, según el problema físico que estemos estudiando. Por ejemplo, si consideramos el problema de la barra del tutorial del [ejemplo mecánico](https://github.com/rirastorza/Intro2FEM/blob/master/Elementos_finitos_en_1D/mecanica1D.ipynb), entonces hacemos $a=AE$, donde $A$ es la sección de la barra y $E$ es el módulo de elasticidad. Por el momento, supondremos que $a$ **es constante**, entonces podemos decir que es [**la ecuación de Poisson**](https://es.wikipedia.org/wiki/Ecuaci%C3%B3n_de_Poisson).


## Formulación variacional

Reescribiremos la Ec. (1) en la forma denominada **forma variacional**. Esto se realiza multiplicando la ecuación por una función de prueba $v$ (**test function**). 

$$-\int_{0}^{1}au^{''} \ \ v \ \ dx = \int_{0}^{1}f \ \ v \ \ dx \tag{2}$$

Bajamos el orden de esta derivada, tal como hicimos en el Problema de Dirichlet. Para esto se integra por partes, si recuerdan como era:

$$\int_{x_{1}}^{x_{2}}f_{1}f_{2}^{'}dx = \left. f_{1}f_{2} \ \ \right |_{x_{1}}^{x_{2}}- \int_{x_{1}}^{x_{2}}f_{1}^{'}f_{2}dx \tag{3},$$

donde llamamos $f_{1} = v$ y $f_{2} = au^{'}$. Operando un poco, como lo hicimos antes, se llega a:

$$\int_{0}^{1}v^{'}au^{'}dx + \underbrace{au^{'}(0)}_{k_{0}\left(u\left(0\right)-g_{0}\right)}v(0) \underbrace{- au^{'}(1)}_{k_{1}\left(u\left(1\right)-g_{1}\right)}v(1) = \int_{0}^{1} fvdx.\tag{4}$$

El dominio $V_{h}$ al que pertenecen la función de prueba ($v$) no puede ser cero en los bordes como en el Problema de Dirichlet porque no podríamos cumpir las condiciones de contorno. Además, $u$ tampoco es cero en los bordes, por lo tanto, debemos conservar la primer y última función sombrero. Entonces:

$$ u_{h} = \sum_{j=0}^{n} \xi_{j} \varphi_{j} \tag{5}.$$

recordemos que antes comenzaba de $j=1$ y finalizaba en $j=n-1$.

## Aproximación por elementos finitos

El siguiente paso es aproximar $u$ por $u_{h}$ de la ecuación anterior y expresarlo como un sistema de ecuaciones lineales del tipo:

$$\left(A+R\right)\xi = b+r \tag{6}.$$

Aparecen ahora una matriz más $R$ y un vector columna más $r$ y las matrices y vectores cambiarán de dimensión, ahora serán de $\left(n+1\right)\times \left(n+1\right)$ y $\left(n+1\right)\times 1$, respectivamente.

### Sistema de ecuaciones lineales

Para construir el sistema de ecuaciones, tomaremos la Ec. (4) por cada parte:

$$\underbrace{\int_{0}^{1}v^{'}au^{'}dx}_{\text{Parte 1}} + \underbrace{k_{0}\left(u\left(0\right)-g_{0}\right)v(0)+ k_{1}\left(u\left(1\right)-g_{1}\right)v(1)}_{\text{Parte 2}} = \underbrace{\int_{0}^{1} fvdx}_{\text{Parte 3}}.\tag{7}$$

- Para la **Parte 1**, reemplazamos $u$ por $u_{h}$ para cada una de las funciones de prueba $\varphi_{i}$ (ahora con $i=0,1,\cdots, n$):

$$ \int_{0}^{1}au_{h}^{'}\varphi_{i}^{'}(x) \ \ dx = \int_{0}^{1}a\sum_{j=0}^{n} \xi_{j} \varphi_{j}^{'}(x)\varphi_{i}^{'}(x) \ \ dx \ \ \text{ con } i =i=0,1,\cdots, n.$$

Por lo tanto los elementos de la matriz $A_{ij}$ serán:

$$ A_{ij} = \int_{0}^{1} a\varphi_{j}^{'} \varphi_{i}^{'} dx \ \ \text{con} \ \ i,j=0,\dots, n \tag{8}$$

- Para la **Parte 2** solo debemos reemplazar:

$$ k_{0}\left(u_{h}(0)-g_{0} \right)\varphi_{i}(0)+k_{1}\left(u_{h}(1)-g_{1} \right)\varphi_{i}(1) = k_{0}\left(\sum_{j=0}^{n} \xi_{j} \varphi_{j}(0)-g_{0} \right)\varphi_{i}(0)+k_{1}\left(\sum_{j=0}^{n} \xi_{j} \varphi_{j}(1)-g_{1} \right)\varphi_{i}(1) \tag{9} $$

Aquí aparecen los elementos de la matriz $R$ y el vector columna $r$:

$$R_{ij} = k_{0}\varphi_{j}(0)\varphi_{i}(0) +  k_{1}\varphi_{j}(1)\varphi_{i}(1) \tag{10}$$

esto es así porque todos los términos de la Ec. (10) están multiplicados por $\xi_{j}$. Noten que $i,j = 0,1,\cdots, n$. Por otro lado, los componentes de $r$ es todo lo que no está multiplicado por $\xi_{j}$.

$$r_{i}=k_{0}g_{0}\varphi_{i}(0)+k_{1}g_{1}\varphi_{i}(1)\tag{11}$$

donde hemos cambiado de signo porque pusimos este vector del lado derecho de la Ec. (6).

- Con la **Parte 3** completamos la expresión:

$$b_{i}=\int_{0}^{1} f \varphi_{i} \ \ dx \ \ \text{con} \ \ i=0,\dots, n\tag{12}$$

De manera similar al caso del Problema de Dirichlet, las incógnitas son los $\xi_{j}$:

$$ \left(\begin{bmatrix} A_{00} & A_{01} & \cdots & A_{0n} \\ A_{10} & \ddots & \cdots & \vdots \\ \vdots &  &  & \vdots \\  A_{n0} & \cdots & \cdots & A_{nn} \end{bmatrix} + \begin{bmatrix} R_{00} & R_{01} & \cdots & R_{0n} \\ R_{10} & \ddots & \cdots & \vdots \\ \vdots &  &  & \vdots \\  R_{n0} & \cdots & \cdots & R_{nn} \end{bmatrix} \right) \begin{bmatrix} \xi_{0} \\ \xi_{1} \\ \vdots \\ \xi_{n} \end{bmatrix}= \begin{bmatrix} b_{0} \\ b_{1} \\ \vdots \\ b_{n} \end{bmatrix} +\begin{bmatrix} r_{0} \\ r_{1} \\ \vdots \\ r_{n} \end{bmatrix} \tag{13}$$

Debe remarcarse que la **matriz de rigidéz** se ha modificado y ahora es $ A + R$ como así también el **vector de cargas** que ahora es $b+r$.

***

Concluyendo, el **algoritmo** deberá realizar las siguientes operaciones:

1. Crear la malla en $I$, esto determinará el espacio $V_{h}$.
2. Computar los coeficientes de $A$, $R$, $r$, y $b$  con las Ecs. (8), (10) (11), y (12) y armar las matrices y vectores.
3. Resolver el sistema de ecuaciones (Ec. (13)).
4. Calcular la solución aproximada: $u_{h}= \sum_{j=0}^{n} \xi_{j} \varphi_{j}$.

***

### Implementamos el algoritmo

Recordemos que las funciones sombrero se definen de la forma:


$$ \varphi_{j}(x_{i})= \left \{ \begin{array}{l} \frac{x-x_{i-1}}{h_{i}} \ \ \text{ si } \ \ x\in I_{i} \\  \frac{x_{i+1}-x}{h_{i+1}}  \ \ \text{ si }  \ \ x\in I_{i+1}\\ 0  \ \ \text{caso contrario} \end{array} \right .$$

entonces su derivada será:

$$ \varphi^{'}_{j}(x_{i})= \left \{ \begin{array}{l} \frac{1}{h_{i}} \ \ \text{ si } \ \ x\in I_{i} \\  \frac{-1}{h_{i+1}}  \ \ \text{ si }  \ \ x\in I_{i+1}\\ 0  \ \ \text{caso contrario} \end{array} \right .$$

Ahora veremos que valores toma el producto $\varphi_{i}^{'}\varphi_{j}^{'}$.

Si la malla es uniforme y el tamaño es $h$, de lo anterior podemos concluir:

1. Para $i=j$:

$$ A_{ii} = \int_{0}^{1} a\left(\varphi_{i}^{'}\right)^{2} dx = \frac{2a}{h} $$

salvo para los bordes, es decir, $A_{00}$ y $A_{nn}$, porque en estos casos se debe integrar la mitad del sombrero. Entonces, para estos casos vale la mitad: $A_{00}=A_{nn}=\frac{a}{h}$.

2. Para $j = i+1$

$$ A_{i(i+1)} = \int_{0}^{1} a\varphi_{i}^{'}\varphi_{i+1}^{'} dx =\int_{x_{i}}^{x_{i+1}} a\varphi_{i}^{'}\varphi_{i+1}^{'} dx = \int_{x_{i}}^{x_{i+1}} a(\frac{-1}{h})\frac{1}{h} dx = \frac{-a}{h} $$

3. En los demás casos vale cero.


Por lo que la matriz A queda:

$$ A = \begin{bmatrix} \frac{a}{h} & \frac{-a}{h} & \cdots & 0 \\ \frac{-a}{h} & \frac{2a}{h} & \cdots & \vdots \\ \vdots &  & \frac{2a}{h} & \vdots \\  0 & \cdots & \frac{-a}{h} & \frac{a}{h} \end{bmatrix}$$

La matriz $R$, como $\varphi_{i}(0) = 1$ solo para $i=0$ y $\varphi_{i}(1) = 1$ para $i=n$ (para todos los demás valores valen 0), quedará: 

$$R = \begin{bmatrix}k_{0} & 0 & \cdots & 0 \\ 0 & \ddots & \cdots & \vdots \\ \vdots &  &  & \vdots \\  0 & \cdots & \cdots & k_{1} \end{bmatrix}$$

Con $r$ ocurre algo similar a lo que pasa con $R$, por lo tanto queda:

$$r= \begin{bmatrix} k_{0}g_{0}\\0\\ \vdots \\ 0\\ k_{1}g_{1}\end{bmatrix}$$

Ahora veamos los $b_{i}$, deberíamos aproximar la Ec. (12), por ejemplo, por una integración por el **método trapezoidal**. En ese caso nos quedaría:

$$b_{i}= \int_{x_{i-1}}^{x_{i}}f\varphi_{i} dx +\int_{x_{i}}^{x_{i+1}}f\varphi_{i} dx = \left[f(x_{i-1})\varphi_{i}(x_{i-1})+f(x_{i})\varphi_{i}(x_{i})\right]\frac{h}{2} +\left[f(x_{i})\varphi_{i}(x_{i})+f(x_{i+1})\varphi_{i}(x_{i+1})\right]\frac{h}{2} = f(x_{i})h$$

para $i=1,2, \cdots, n-1$, pero para $i=0$ e $i=n$ solo integramos la mitad. Entonces el vector $b$ queda:

$$ b = \begin{bmatrix} f(x_{0})\frac{h}{2}\\f(x_{1})h\\ \vdots \\ f(x_{n-1})h \\ f(x_{n})\frac{h}{2} \end{bmatrix}.$$

De aquí en adelante podemos continuar y desarrollar este script con Python tal como lo hicimos en el **ejemplo5.py** (esto queda como tarea para ustedes).

## Parámetros variables

Un comentario aparte merece la posibilidad de hacer los parámetros variables también con la posición. Continuando con el ejemplo mecánico, se puede pensar en una barra cuya sección no sea constante ($A$ es variable) o que cambien el material (en cuyo caso $E$ sería variable). Este caso sería similar a lo que hemos estudiado, pero ya no será una ecuación de Poisson, y debemos expresar $-\left(au^{'}\right)^{'}=f $. Tomemos el caso sección variable ($a= A(x)E$) dependiente de la posición. 

[<img src="barravariable.png" width="200"/>](barravariable.png)

En la figura, se muestra como se puede generar la malla, y como aproximar la función $a(x)$. En este caso consideramos la sección del intervalo $\left[x_{i-1},x_{i}\right]$ como el área en el punto medio, esto es, $A_{i}$, entonces $a_{i} = A_{i}E$.

Por lo tanto el elemento de la matriz $A$ será:

$$ A_{ii} = \int_{0}^{1} a\left(\varphi_{i}^{'}\right)^{2} dx \approx a_{i}\frac{h_{i}}{h^{2}_{i}} +a_{i+1}(-1)^{2}\frac{h_{i+1}}{h^{2}_{i+1}}=\frac{a_{i}}{h_{i}}+\frac{a_{i+1}}{h_{i+1}} \ \ \text{ para } i=1,2,\cdots, n-1$$

Para el caso de $A_{i, i+1}$:

$$A_{i,i+1} = \int_{0}^{1} a\varphi_{i}^{'}\varphi_{i+1}^{'} dx = \int_{x_{i}}^{x_{i+1}} a\varphi_{i}^{'}\varphi_{i+1}^{'} dx \approx a_{i+1}\frac{-1}{h_{i+1}}\frac{1}{h_{i+1}}h_{i+1}=-\frac{a_{i+1}}{h_{i+1}}$$